In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib

In [2]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver',options=options)

In [3]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup


#Checks if bio_url is a valid faculty homepage
def is_valid_homepage(bio_url,dir_url):
    try:
        #sometimes the homepage url points to the faculty profile page
        #which should be treated differently from an actual homepage
        ret_url = urllib.request.urlopen(bio_url).geturl() 
    except:
        return False       #unable to access bio_url
    urls = [re.sub('((https?://)|(www.))','',url) for url in [ret_url,dir_url]] #removes url scheme (https,http) or www 
    return not(urls[0]== urls[1])

In [38]:
#extracts all Faculty Profile page urls from the Directory Listing Page
def scrape_dir_page(dir_url,browser):
    print ('-'*20,'Scraping directory page','-'*20)
    faculty_links = []
    faculty_base_url = 'https://www.imdb.com'
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,browser)     
    
    #url returned is relative, so we need to add base url
    faculty_links.append(faculty_base_url+soup.find_all('td',class_="result_text")[0].find('a')['href'])
    mainpage = faculty_links[0]
    soup2 = get_js_soup(mainpage,browser)

    faculty_links.append(faculty_base_url+soup2.find_all("div",class_ = "poster")[0].find("a")["href"])

    page3 = faculty_links[1]
    soup3 = get_js_soup(page3,browser)
   # print(type(soup3.find_all("img",class_ = "pswp__img")[0]))
    faculty_links.append(faculty_base_url+soup3.find_all("div",class_ = "pswp__item")[1].find_all("img",class_ = "pswp__img")[1].find("src"))
   
    print(faculty_links)
    return faculty_links

In [98]:
def generate_url(query):
    base = "https://www.imdb.com/find?ref_=nv_sr_fn&q="
    for i in query:
        base+=i
        print(base)
        base+="+"
        print(base)
    base+="&s=all"
    return base



In [97]:
dir_url = "https://www.imdb.com/find?ref_=nv_sr_fn&q=avengers+age+of+ultron+&s=all" #url of directory listings of CS faculty
faculty_links = scrape_dir_page(dir_url,browser)


['https://www.imdb.com/title/tt2395427/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMTM4OGJmNWMtOTM4Ni00NTE3LTg3MDItZmQxYjc4N2JhNmUxXkEyXkFqcGdeQXVyNTgzMDMzMTg@._V1_SY1000_SX675_AL_.jpg', '7.3']
